### This code extracts data about 1) spike counts for each trial, separated into correct/incorrect contralateral, ipsilateral, and nogo 2) percent corrent choices for each session of behavior and 3) average time from stimulus presentation to auditory GO cue.

In [2]:
## to run this code, you need to have the behavioral data dictionaries previously loaded in
## see 'loading_in_tar' and 'loading_in_npy' files in repo, limited code copied below:

# os.chdir('/your/filepath/here')
# goCue_times = np.load("goCue_times.npy", allow_pickle=True)
# response_choices = np.load('response_choices.npy', allow_pickle=True)
# feedback_types = np.load('feedback_types.npy', allow_pickle=True)
# stim_times = np.load('stim_times.npy', allow_pickle=True)

In [ ]:
## get trial types (contra/ipsi/nogo) for a given session
## calculate percent correct for each session

perc_corr = np.zeros(40)
sessions = np.arange(1, 40)
trial_type = {new_key: [] for new_key in sessions}

contra_corr_SC = []
contra_corr_nBins = []
contra_incorr_SC = []
contra_incorr_nBins = []

ipsi_corr_SC = []
ipsi_corr_nBins = []
ipsi_incorr_SC = []
ipsi_incorr_nBins = []

nogo_SC = []
nogo_nBins = []

for s in sessions:
    print('session', s)
    trials = len(goCue_times[s]) # nTrials
    contra_Ctrials = np.zeros(trials) # tracking trials where correct response is contraleteral to probe
    contra_Itrials = np.zeros(trials) # tracking trials where INcorrect response is contraleteral to probe
    ipsi_Ctrials = np.zeros(trials) # tracking trials where correct response is ipsileteral to probe
    ipsi_Itrials = np.zeros(trials) # tracking trials where correct response is ipsileteral to probe
    nogo_trials = np.zeros(trials) # tracking NoGo trials
    
    for t in range(trials):
        response = int(response_choices[s][t]) # -1 right, 0 nogo, 1 left
        feedback = int(feedback_types[s][t]) # -1 incorrect, 1 correct
        if response == 1 and feedback == 1:
            contra_Ctrials[t] = 1
        elif response == 1 and feedback == -1:
            contra_Itrials[t] = 1
        elif response == -1 and feedback == 1:
            ipsi_Ctrials[t] = 1
        elif response == -1 and feedback == -1:
            ipsi_Itrials[t] = 1
        elif response == 0:
            nogo_trials[t] = 1
    
    pc = np.count_nonzero(feedback_types[s]==1)/(len(feedback_types[s]))
    perc_corr[s] = pc
    trial_type[s] = np.stack((contra_Ctrials, contra_Itrials, ipsi_Ctrials, ipsi_Itrials, nogo_trials), axis=0)
    
    ## now that we know which trials are which type, we need timestamps for each of those trials to get FR
    time_window = []
    
    for t in range(trials):

        lowerT = stim_times[s][t]
        upperT = goCue_times[s][t]
        this_trial = []
        
        for key,value in vis_timestamps.items():
            if s == key:
                all_stamps = np.arange(1, len(vis_timestamps[s]))
                for i in all_stamps:
                    if lowerT <= vis_timestamps[s][i] < upperT: 
                        this_trial.append(vis_timestamps[s][i])
                        
        time_window.append(upperT-lowerT)

        if trial_type[s][0][t] == 1: ## if this contra trial was correct
            contra_corr_SC.append(len(this_trial))
            contra_corr_nBins.append((upperT-lowerT)//0.10)

        elif trial_type[s][0][t] == 1: ## if this contra trial was incorrect
            contra_incorr_SC.append(len(this_trial))
            contra_incorr_nBins.append((upperT-lowerT)//0.10)

        elif trial_type[s][2][t] == 1: ## if this ipsi trial was correct
            ipsi_corr_SC.append(len(this_trial))
            ipsi_corr_nBins.append((upperT-lowerT)//0.10)

        elif trial_type[s][3][t] == 1: ## if this ipsi trial was incorrect
            ipsi_incorr_SC.append(len(this_trial))
            ipsi_incorr_nBins.append((upperT-lowerT)//0.10)

        elif trial_type[s][4][t] == 1: ## if this was a nogo trial
            nogo_SC.append(len(this_trial))
            nogo_nBins.append((upperT-lowerT)//0.10)

In [ ]:
## helper function
def divide_two_lists(list1, list2):
    new_list = []
    for x1 in list1:
        for x2 in list2:
            new_list.append((x1/x2))
return new_list

In [ ]:
## average time window and % correct
print('avg time window:', np.mean(time_window))
print('avg % correct trials:', np.mean(perc_corr))

## get spike counts per bin for each trial type
cc = divide_two_lists(contra_corr_SC/contra_corr_nBins)
print('average spikes/bin in contra - correct trials is', np.mean(cc))
ci = divide_two_lists(contra_incorr_SC/contra_incorr_nBins)
print('average spikes/bin in contra - incorrect trials is', np.mean(ci))
ic = divide_two_lists(ipsi_corr_SC/ipsi_corr_nBins)
print('average spikes/bin in ipsi - correct trials is', np.mean(ic))
ii = divide_two_lists(ipsi_incorr_SC/ipsi_incorr_nBins)
print('average spikes/bin in ipsi - incorrect trials is', np.mean(ii))
ng = divide_two_lists(nogo_SC/nogo_nBins)
print('average spikes/bin in nogo trials is', np.mean(ng))